<a href="https://colab.research.google.com/github/patchikoooo/data-science-from-scratch/blob/master/ConfidenceIntervals_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Confidence Intervals**

In [0]:
import math
import random

In [0]:
def normal_cdf(x, mu=0, sigma=1):
    return (1 + math.erf((x-mu) / math.sqrt(2) / sigma)) / 2

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    # find approximate inverse using binary search
    
    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z, low_p = -10.0, 0          # normalcdf(-10) is (very close to) 0
    hi_z, hi_p = 10.0, 1             # normal_cdf(10) is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2   # consider the midpoint
        mid_p = normal_cdf(mid_z)    # and the cdf's value there
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break
    
    return mid_z

normal_probability_below = normal_cdf

def normal_approximation_to_binomial(n, p):
    # finds mu and sigma corresponding to Binomial(n, p)
    mu = p*n
    sigma = math.sqrt(n*p*(1-p))
    return mu, sigma

In [0]:
# we can use normal_cdf to figure out that probability that its realized value
#    lies within (or outside) a particular interval

# the normal cdf is the probability the variable is below a threshold
normal_probability_below = normal_cdf

# it's above the threshold if it's not below the threshold
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# it's between if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# it's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu=0, sigma=1)

In [0]:
# We can also do reverse -- find either the nontail region or the (symmetric) interval
#   around the mean that accounts for a certain level of likelihood.

# For example, if we want to find an interval centered at the mean and containing
    # 60% probability, then we find the cutoffs where the upper and lower tails
    # each contain 20% of the probability( leaving 60%)


# z --> z score
def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z)"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds that contain
        the specified probability"""
    tail_probability = (1 - probability) / 2

    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # lower bound should have tail probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less thatn the mean, the tail is what's less than x
        return 2 * normal_probability_below(x, mu, sigma)

In [0]:
""" For example, we can estimate the probability of the unfair coin by looking
        at the average value of the Bernoulli variables corresponding to each flip.
        If we observe 525 heads out of 1,000 flips, then we can estimate the
        observed value of the parameter.
    
    How confident can we be about this estimate? Weell, if we knew the exact
        value of p, CLT tells us that the average of those Bernoulli variables
        should be approximately normal, with mean p and standard deviation """

math.sqrt(p * (1-p) / 1000)

NameError: ignored

In [0]:
# since we don't know our p, we use p_hat
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print("(mu, sigma)", (mu, sigma))

# This is not entirely justified, but people seem to do it anyway.
#   Using the normal approximation, we conclude that we are "95% confident"
#   that the folloowing interval contains the true parameter p.

normal_two_sided_bounds(0.95, mu, sigma)

(mu, sigma) (0.525, 0.015791611697353755)


(0.4940490278129096, 0.5559509721870904)

In [0]:
# trying 540 heads out of 1000, we'd have:

p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print("(mu, sigma)", (mu, sigma))
normal_two_sided_bounds(0.95, mu, sigma)

# Here, "fair coin" doesn't lie in the confidence interval

(mu, sigma) (0.54, 0.015760710643876435)


(0.5091095927295919, 0.5708904072704082)

# **P-hacking**

In [0]:
""" A procedure that erroneously rejects the null hypothesis only 5% of the time
        - by definition - 5% of the time erroneously reject the hypothesis """

def run_experiment():
    """ flip a fair coin 1000 times, True = heads, False = tails """
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    """ using the 5% significance levels """
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

In [0]:
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment for experiment in experiments
                      if reject_fairness(experiment)])
num_rejections

46

In [0]:
""" Running AB Test """

""" One of your primary responsibilities at DataSciencester is experience optimization,
which is a euphemism for trying to get people to click on advertisements.

One of your advertisers has developed a new energy drink targeted at data scientists,
    and the VP of Advertisements wants your help choosing between advertisement A
    (“tastes great!”) and advertisement B (“less bias!”). Being a scientist,
    you decide to run an experiment by randomly showing site visitors one
    of the two advertisements and tracking how many people click on each one.
    
    If 990 out of 1,000 A-viewers click their ad while only 10 out of 1,000
    B-viewers click their ad, you can be pretty confident that A is the better
    ad. But what if the differences are not so stark? Here’s where you’d use
    statistical inference. """

' One of your primary responsibilities at DataSciencester is experience optimization,\nwhich is a euphemism for trying to get people to click on advertisements.\n\nOne of your advertisers has developed a new energy drink targeted at data scientists,\n    and the VP of Advertisements wants your help choosing between advertisement A\n    (“tastes great!”) and advertisement B (“less bias!”). Being a scientist,\n    you decide to run an experiment by randomly showing site visitors one\n    of the two advertisements and tracking how many people click on each one.\n    \n    If 990 out of 1,000 A-viewers click their ad while only 10 out of 1,000\n    B-viewers click their ad, you can be pretty confident that A is the better\n    ad. But what if the differences are not so stark? Here’s where you’d use\n    statistical inference. '

In [0]:
def estimated_parameters(N, n):
    # n is the number of successes and N is the total number of trials
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

In [0]:
estimated_parameters(1000, 800)

(0.8, 0.012649110640673518)

In [0]:
""" If we assume those two normals are independent, then their difference should
        also be normal with mean Pb - Pa and standard deviation
        math.sqrt(sigmaA^2 + sigmaB^2)

    Note that this is sort of CHEATING. The math only works out exactly like this
        if you know the standard deviations. Here we're estimating them from the data,
        which means that we really should be using a t-distribution. But for large
        enough data sets, it's close enought that it doesn't make much of a
        difference.

    This means we can test the null hypothesis that Pa and Pb are the same.
        that is, that Pa - Pb == 0;
        using the statistic: """

def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    # mean difference and sigma difference
    # sigma difference will also be sum since squaring sigma will yield
    #       positive number
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

    # this should be approximately be a standard normal.

In [0]:
""" For example, if "tastes great" gets 200 clicks our of 1000 views and "less
        bias" gets 180 clicks out of 1000 views, the statistic equals: """

z = a_b_test_statistic(1000, 200, 1000, 180)
print(z)

-1.1403464899034472


In [0]:
""" The probability of seeing such a large difference if the means 
        were actually equal would be: """



# Which is large enough that you can't conclude there's much of a difference.



0.254141976542236


In [0]:
# On the other hand, if "less bias" only got 150 chicks, we'd have:
z = a_b_test_statistic(1000, 200, 1000, 150)
print(two_sided_p_value(z))

#   which means there's only a 0.003 probability you'd see such a large
#   if the ads were equally effective.

0.003189699706216853


# **Bayesian Inference**

In [0]:
"""
The procedures we've looked at have involved making probability statements about
    our tests: "there's only a 3% chance you'd observe such an extreme statistic
    if our null hypothesis were true.
"""

""" An  alternative approach to inference involves treating the unknown parameters
        themselves as random variables. The analyst ( that's you ) starts with a
        prior distribution for the parameters and then uses the observed data
        and Baye's Theorem to get an updated posterioir distribution for the
        parameters. Rather than making probability judgments about the tests,
        you make probability judgments about the parameters themselves. """

# For example, when unknown parameter is a probability, we often use a prior
#   from the Beta distribution, which puts all its probability between 0 and 1:

def B(alpha, beta):
    # normalizing constant so that the total probability is 1
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:   # no weight outside of [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (betta - 1) / B(alpha, beta)

# this distribution centers its weight at:
#       alpha / (alpha + beta)
#       and the larger alpha and beta are, the "tighter" the distribution is


In [0]:
"""
For example, if alpha and beta are both 1, it's just the uniform distribtion
    (centered at 0.5, very dispered). If alpha is much larger than beta,
    most of the weights is near 1. And if alpha is much smaller than beta,
    most of the weight is near zero.
"""

""" Let's assume a priod distribution on p. Maybe we don't want to take
        a stand on whether the coin if fair, and we choose alpha and beta
        to both equal 1. Or maybe we have a strong belief that it lands heads
        55% of the time, and we choose alpha equals 55, beta equals 45.

        Then we flip our coin a bunch of times and see h heads and t tails.
        Baye's Theorem ( and some mathematics that's too tedious for us to go
        through here) tells us that the posterioir distribution for p is again
        a Beta distribution with parameters alpha + h and beta + t. """

""" NOTE:
It is coincidence that the posterior distribution was again a Beta distribution.
    The number of heads given by a Binomial distribution, and the Beta is the 
    conjugate prioir to the binomial distribution. This means that whenever you
    update a Beta prior using observations from the corresponding binomial, you
    will get back a Beta posterior.
"""

' NOTE:\nIt is coincidence that the posterior distribution was again a Beta distribution.\n    The number of heads given by a Binomial distribution, and the Beta is the \n    conjugate prioir to the binomial distribution. This means that whenever you\n    update a Beta prior using observations from the corresponding binomial, you\n    will get back a Beta posterior.\n'

In [0]:
# end page 142

**For Further Exploration**



*   We've barely scratched the surface of what you should know about statistical inference. The books recommended at the end of Chapter 5 go into a lot more detail.
*   Coursera offers a [Data Analysis and Statistical](https://www.coursera.org/specializations/statistics) Inference course that covers many of these topics.

